# PyTorch - homework 2: neural networks

-- Prof. Dorien Herremans

Please run the whole notebook with your code and submit the `.ipynb` file on eDimension that includes your answers [so after you run it]. 

In [1]:
from termcolor import colored

student_number="1003056"
student_name="Ivan Christian"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Ivan Christian, number: 1003056


 ## Question 1 -- XOR neural network [3pts]

a) Train an (at least) 2-layer neural network that can solve the XOR problem. 

b) Check the predictions resulting from your model in the second code box below.


In [2]:
# load your data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

import numpy as np

X = torch.Tensor([[0,0],[0,1], [1,0], [1,1]])
Y = torch.Tensor([[0,1,1,0]]).view(-1,1)

# name your model xor


class xor(nn.Module):
    def __init__(self, input_dim = 2, output_dim=1):
        super(xor, self).__init__()
        self.lin1 = nn.Linear(input_dim, 512)
        self.lin2 = nn.Linear(512, output_dim)
    
    def forward(self, x):
        x = self.lin1(x)
        x = F.sigmoid(x)
        x = self.lin2(x)
        x = F.sigmoid(x)
        return x

model = xor().cuda()

# define your model loss function, optimizer, etc. 
device = 'cuda'
loss_func = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)



def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)
weights_init(model)

# train the model
def train(x, y, model, loss_func, optimizer, device, epochs=8001):
    steps = x.size(0)
    for epoch in range(epochs):
        for j in range(steps):
            data_point = np.random.randint(x.size(0))
            x_var = Variable(x[data_point], requires_grad=False).to(device)
            y_var = Variable(y[data_point], requires_grad=False).to(device)
            optimizer.zero_grad()
            y_hat = model(x_var)
            loss = loss_func(y_hat, y_var)
            loss.backward()
            optimizer.step()
        if epoch % 500 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.cpu().data.numpy()}, ")
train(X, Y, model, loss_func, optimizer, device)


C:\Users\chris\Anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0, Loss: 0.9997462034225464, 
Epoch: 500, Loss: 0.0002214344567619264, 
Epoch: 1000, Loss: 0.00012036479165544733, 
Epoch: 1500, Loss: 8.845984120853245e-05, 
Epoch: 2000, Loss: 6.330659380182624e-05, 
Epoch: 2500, Loss: 8.215211710194126e-05, 
Epoch: 3000, Loss: 0.9999990463256836, 
Epoch: 3500, Loss: 3.3376607461832464e-05, 
Epoch: 4000, Loss: 2.8901149562443607e-05, 
Epoch: 4500, Loss: 0.9999994039535522, 
Epoch: 5000, Loss: 0.9999995231628418, 
Epoch: 5500, Loss: 2.112330730597023e-05, 
Epoch: 6000, Loss: 1.9251523553975858e-05, 
Epoch: 6500, Loss: 1.9294542653369717e-05, 
Epoch: 7000, Loss: 2.9500253731384873e-05, 
Epoch: 7500, Loss: 1.5546536815236323e-05, 
Epoch: 8000, Loss: 1.4926215953892097e-05, 


In [3]:
# test your model using the following functions (make sure the output is printed and saved when you submit this notebook):
# depending on how you defined your network you may need to slightly tweek the below prediction function

test = [[0,0],[0,1], [1,0], [1,1]]


for trial in test: 
    Xtest = torch.Tensor(trial).to(device)
    model = xor().to(device)
    y_hat = model(Xtest).cuda()
    prediction = 1 if y_hat[0] > 0.5 else 0
    print("{0} xor {1} = {2}".format(int(Xtest[0]), int(Xtest[1]), prediction))


0 xor 0 = 1
0 xor 1 = 1
1 xor 0 = 0
1 xor 1 = 0


## Question 2  [2pts]

Imagine a neural network model for a multilabel classification task. 

a) Which loss function should you use?

b) The resulting trained modal has a high variance error. Give 4 possible solutions to improve the model. 


```
* answer A
a) Binary cross entropy loss
* answer B
  - 1
  - 2
  - ...
b) High Variance Error Possible (Overfitting) solutions:
- 1 : Dropout 
- 2 : Data augmentation during training time
- 3 : Early Stopping
- 4 : Get more training data

```


## Question 3 - Improve hit classification [5pts]

Remember the hit predicton dataset from last week? 

a) Improve the model using a multiplayer perceptron. 

b) Make sure to run your models on the GPU. 

c) Tweek the hyperparameters such as number of nodes or layers, or other. Show two possible configurations and explain which works better and very briefly explain why this may be the case. 




In [4]:
# code your model 1
# Perceptron structure --> fc -> relu

!pip install wget
import wget
import os

import pandas as pd 

train_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv'
test_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv'


if not os.path.exists('herremans_hit_1030training.csv') and not os.path.exists('herremans_hit_1030test.csv'):
    train_data = wget.download(train_url) 
    test_data = wget.download(test_url)
else:
    train_data = 'herremans_hit_1030training.csv'
    test_data = 'herremans_hit_1030test.csv'

# load data

train_data = pd.read_csv(train_data)
test_data = pd.read_csv(test_data)

# define logistic regression model

import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
  # input_size: Dimensionality of input feature vector.
  # num_classes: The number of classes in the classification problem.
    def __init__(self, input_size, num_classes):
    # Always call the superclass (nn.Module) constructor first!
        super(LogisticRegression, self).__init__()
        self.l1 = nn.Linear(input_size, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, num_classes)
    def forward(self, x):
        out = self.l1(x)
        out = torch.relu(out)
        out = self.l2(out)
        out = torch.relu(out)
        out = self.l3(out)
        out = torch.relu(out)
        out = torch.sigmoid(out)
        return out

# train model

device = 'cuda'
epochs = 9001

num_outputs = 1
num_input_features = 49

lr_rate = 0.005
loss_function = nn.BCELoss()
model1 = LogisticRegression(num_input_features, num_outputs).to(device)
optimizer = torch.optim.SGD(model1.parameters(), lr=lr_rate)

for epoch in range(epochs):
    x_data = torch.FloatTensor(train_data.loc[:, train_data.columns != 'Topclass1030'].values).to(device)
    y_true = torch.FloatTensor(train_data['Topclass1030']).to(device)
    
    optimizer.zero_grad()
    
    y_pred = model1(x_data)
    
    loss = loss_function(y_pred, y_true).to(device) #calculate the loss
    loss.backward() #backprop
    optimizer.step()
    if epoch % 500 == 0:
        print ("Epoch: {0}, Loss: {1}, ".format(epoch,loss))

C:\Users\chris\Anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([321])) that is different to the input size (torch.Size([321, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0, Loss: 0.6820421814918518, 
Epoch: 500, Loss: 0.6343249082565308, 
Epoch: 1000, Loss: 0.6017062664031982, 
Epoch: 1500, Loss: 0.5698546171188354, 
Epoch: 2000, Loss: 0.5402866005897522, 
Epoch: 2500, Loss: 0.5157894492149353, 
Epoch: 3000, Loss: 0.4952634871006012, 
Epoch: 3500, Loss: 0.4757671058177948, 
Epoch: 4000, Loss: 0.45520785450935364, 
Epoch: 4500, Loss: 0.43378180265426636, 
Epoch: 5000, Loss: 0.4119996130466461, 
Epoch: 5500, Loss: 0.39112672209739685, 
Epoch: 6000, Loss: 0.3720206320285797, 
Epoch: 6500, Loss: 0.3553045094013214, 
Epoch: 7000, Loss: 0.34097975492477417, 
Epoch: 7500, Loss: 0.32933303713798523, 
Epoch: 8000, Loss: 0.3203296959400177, 
Epoch: 8500, Loss: 0.31262582540512085, 
Epoch: 9000, Loss: 0.30709490180015564, 


In [5]:
# evaluate model 1 (called model1 here)


import pandas as pd 

def run_evaluation(my_model):
    '''
    Function to evaluate the performance of the custom model
    
    Args:
    - my_model : model (custom model to be tested)
    returns:
    - None
    '''

    device = 'cuda'
    my_model.eval()

    test = pd.read_csv('herremans_hit_1030test.csv')
    labels = test.iloc[:,-1]
    test = test.drop('Topclass1030', axis=1)
    testdata = torch.Tensor(test.values).to(device)
    testlabels = torch.Tensor(labels.values).view(-1,1).to(device)

    TP = 0
    TN = 0
    FN = 0
    FP = 0



    for i in range(0, testdata.size()[0]): 
        Xtest = testdata[i]
        y_hat = my_model(Xtest)
        if y_hat > 0.5:
            prediction = 1
        else: 
            prediction = 0

        if (prediction == testlabels[i]):
            if (prediction == 1):
                TP += 1
            else: 
                TN += 1

        else:
            if (prediction == 1):
                FP += 1
            else: 
                FN += 1

    print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
    print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
    rate = TP/(FN+TP)
    print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

run_evaluation(model1)

True Positives: 45, True Negatives: 14
False Positives: 15, False Negatives: 5
Class specific accuracy of correctly predicting a hit song is 0.9


In [6]:
# code your model 2
!pip install wget
import wget
import os

import pandas as pd 

train_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv'
test_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv'


if not os.path.exists('herremans_hit_1030training.csv') and not os.path.exists('herremans_hit_1030test.csv'):
    train_data = wget.download(train_url) 
    test_data = wget.download(test_url)
else:
    train_data = 'herremans_hit_1030training.csv'
    test_data = 'herremans_hit_1030test.csv'

# load data

train_data = pd.read_csv(train_data)
test_data = pd.read_csv(test_data)

# define logistic regression model

import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
  # input_size: Dimensionality of input feature vector.
  # num_classes: The number of classes in the classification problem.
    def __init__(self, input_size, num_classes, dropout=0.6):
    # Always call the superclass (nn.Module) constructor first!
        super(LogisticRegression, self).__init__()
        self.l1 = nn.Linear(input_size, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 64)
        self.l4 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = self.l1(x)
        out = torch.relu(out)

        out = self.l2(out)
        out = torch.relu(out)
        out = self.l3(out)
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.l4(out)
        out = torch.relu(out)
        out = self.dropout(out)
        out = torch.sigmoid(out)
        return out


# train model

device = 'cuda'
epochs = 9001

num_outputs = 1
num_input_features = 49

lr_rate = 0.005
loss_function = nn.BCELoss()
model2 = LogisticRegression(num_input_features, num_outputs).to(device)
optimizer = torch.optim.SGD(model2.parameters(), lr=lr_rate)

for epoch in range(epochs):
    x_data = torch.FloatTensor(train_data.loc[:, train_data.columns != 'Topclass1030'].values).to(device)
    y_true = torch.FloatTensor(train_data['Topclass1030']).to(device)
    
    optimizer.zero_grad()
    
    y_pred = model2(x_data)
    
    loss = loss_function(y_pred, y_true).to(device) #calculate the loss
    loss.backward() #backprop
    optimizer.step()
    if epoch % 500 == 0:
        print ("Epoch: {0}, Loss: {1}, ".format(epoch,loss))

Epoch: 0, Loss: 0.6893376111984253, 
Epoch: 500, Loss: 0.6533893346786499, 
Epoch: 1000, Loss: 0.6706206202507019, 
Epoch: 1500, Loss: 0.6420188546180725, 
Epoch: 2000, Loss: 0.6372660398483276, 
Epoch: 2500, Loss: 0.6195134520530701, 
Epoch: 3000, Loss: 0.6013129949569702, 
Epoch: 3500, Loss: 0.6084885597229004, 
Epoch: 4000, Loss: 0.5967109203338623, 
Epoch: 4500, Loss: 0.5848504900932312, 
Epoch: 5000, Loss: 0.5726922154426575, 
Epoch: 5500, Loss: 0.5605628490447998, 
Epoch: 6000, Loss: 0.5519987940788269, 
Epoch: 6500, Loss: 0.5650646090507507, 
Epoch: 7000, Loss: 0.548926830291748, 
Epoch: 7500, Loss: 0.5453125238418579, 
Epoch: 8000, Loss: 0.5360672473907471, 
Epoch: 8500, Loss: 0.5202764868736267, 
Epoch: 9000, Loss: 0.5404687523841858, 


In [7]:
# evaluate model 2 (called model2 here)

run_evaluation(model2)

True Positives: 47, True Negatives: 13
False Positives: 16, False Negatives: 3
Class specific accuracy of correctly predicting a hit song is 0.94


Which works better and why do you think this may be (very briefly)? 


**[your answer here, also please summarise the differences between your two models]**

Model 1 is a 3-layer multilayer perceptron model

Model 2 is a 4-layer multilayer perceptron model with dropouts after layer 3 and 4. Layers 2 (512 nodes), Layers 3 (256 nodes) and Layers 4 (64 nodes) are upgraded versions of model which only has Layer 2 (128 nodes) and Layer 3 (64 nodes). This makes model 2 a more complicated model hence making it better than the original model.

Model 2 is better because of the following reasons:

- Dropout makes the model more generalisable and it results 
- more layers mean that the model will be able to learn more complex features. As such, there is an increase in the test accuracy when using the model 2.

Additionally, submit your results [here](https://forms.gle/NtJJEE7Wm5ZRM3Je7) for 'Class specific accuracy of correctly predicting a hit song' and see if you got the best performance of the class! Good luck!